In [1]:
import soup
from common import JsonDict, DotDict, SGR, StartDaemonThread, b64enc, b64dec
import json, os, requests, time, traceback

class RequestError(Exception):
    pass

In [2]:
class RequestError(Exception):
    pass

class OneBotApi():
    def __init__(self, token=None, host='localhost', hp=5700, wp=5800, **kwargs) -> None:
        self.started = None
        self.host = host
        self.hp = hp
        self.wp = wp
        self.kwargs = kwargs
        self.headers = {'Authorization':f'Bearer {token}'}
        self.Friend = lambda **kwargs:[t for t in self.List('friend') if not kwargs or all(t[k]==v for k,v in kwargs.items())]
        self.Group = lambda **kwargs:[t for t in self.List('group') if not kwargs or all(t[k]==v for k,v in kwargs.items())]
        self.Member = lambda **kwargs:[t for g in self.Group() for t in self.List('member',g.group_id) if not kwargs or all(t[k]==v for k,v in kwargs.items())]

    def basicsession(self, mode, url, **kwargs):
        kw = kwargs
        kw.update(self.kwargs)
        while True:
            if not self.started:continue
            try:
                print(f'url:http://{self.host}:{self.hp}/{url} kw:{kw}')
                response = getattr(requests, mode)(f'http://{self.host}:{self.hp}/{url}',headers=self.headers , **kw)
                r = DotDict(response.text)
                if response.status_code != 200:raise RequestError(f'status_code: {response.status_code}')
                break
            except requests.exceptions.ConnectionError:
                print('无法连接倒OneBot，请检查服务、地址、端口。')
            except RequestError as e:
                print(e)
            except Exception as e:
                raise RequestError(e)
        if (hasattr(r, 'status') and r.status != 'ok') or (hasattr(r,'retcode') and r.retcode != 0):
            print(f'mode: {mode}, url: {url}, kwargs: {kwargs}, {", ".join([f"{k}: {v}" for k,v in r.items()])}')
            return r
        return r.data

### 消息与事件上报 ###
    def pollForever(self, Analyst): # WS Adapter
        import websocket
        while True:
            try:
                self.ws = websocket.create_connection(f'ws://{self.host}:{self.wp}', header=self.headers, timeout=60)
                recv = self.ws.recv()
                if not recv:raise
                recv = DotDict(recv)
            except:
                if self.started:
                    print('qsession.Poll 方法出错请检查连接配置', exc_info=True)
                    self.started = False
                time.sleep(15)
                continue
            print(f'qq:{recv.self_id} 链接成功')
            self.qq = recv.self_id
            self.started = True
            while self.started:
                try:
                    recv = DotDict(self.ws.recv())
                    if recv.post_type == 'meta_event':
                        print(f'qq.status:{recv.status["qq.status"]}')
                        continue
                    StartDaemonThread(Analyst, recv)
                except Exception as e:
                    print(e)
                    break
                
    def LoginInfo(self):
        '该接口用于获取 QQ 用户的登录号信息。'
        return self.basicsession('get','get_login_info')
    
    def List(self, mode:str, gid:int=None, **kwargs):
        '''该接口用于获取联系人列表。
    mode	string	是	获取对象类型：unfriend|friend|group|member
	gid		int		否	群ID'''
        mode = mode.lower()
        payload = JsonDict()
        if mode == 'unfriend':url = 'get_unidirectional_friend_list'
        elif mode == 'friend':url = 'get_friend_list'
        elif mode == 'group':url = 'get_group_list'
        elif mode == 'member':
            url = 'get_group_member_list'
            payload.group_id = int(gid)
        return self.basicsession('post',url,json=payload)

    def SendMsg(self, mode:str, target:int, *message:object|list, reply:int=0, recall:int=0):
        '''该接口用于发送消息。
    '''
        mode = mode.lower()
        payload = JsonDict({'user_id' if mode == 'friend' else 'group_id':target})
        message = [JsonDict({'type':msg.pop('type'), 'data':msg}) for msg in [JsonDict(msg.copy()) for msg in list(message) + ([soup.Reply(reply)] if reply else [])]]
        if any([msg.type=='node' for msg in message]):
            url = 'send_private_forward_msg' if mode == 'friend' else 'send_group_forward_msg'
            for node in message:
                if 'content' in node.data:node.data.content = [{'type':msg.pop('type'),'data':msg} for msg in [msg.copy() for msg in node.data.content]]
            payload.messages = message
        else:
            url = 'send_msg'
            payload.message_type = 'private' if mode == 'friend' else 'group'
            if recall:payload.recall_duration = recall
            for msg in message:
                if 'data_type' in msg.data:msg.data.type = msg.data.pop('data_type')
            payload.message = message
        data = self.basicsession('post',url,json=payload)
        if mode=="friend":
            target = self.Friend(user_id=target)[0]
            print(f'发到好友{SGR(target.nickname,b4=11)}[{SGR(target.remark,b4=11)}({SGR(target.user_id,b4=1)})]{(reply and "回复("+SGR(reply,b4=2)+")") or ""}消息({SGR(data.message_id, b4=12) if "message_id" in data else SGR(data.retcode, b4=11)}):\n{str(message)}')
        elif mode=="group":
            target = self.Group(group_id=target)[0]
            print(f'发到群{SGR(target.group_name,b4=14)}({SGR(target.group_id,b4=4)}){(reply and "回复("+SGR(reply,b4=2)+")") or ""}消息({SGR(data.message_id, b4=12) if "message_id" in data else SGR(data.retcode, b4=11)}):\n{str(message)}')
        return data


In [3]:
host = input('HOST: ')
hp = input('HTTP_PORT: ')
wp = input('WEBSOCKET_PORT: ')
token = input('TOKEN: ')

In [4]:
bot = OneBotApi(token,host,hp,wp)
bot.started = True
bot.LoginInfo()

url:http://192.168.0.2:9700/get_login_info kw:{}


{'user_id': 2907237958, 'nickname': '精神病院里的人'}

In [ ]:
bot.basicsession('post','send_private_forward_msg',json={
    'user_id':'1064393873',
    'messages':[
        {
            'type':'node',
            'data':{
                'user_id':'1064393873',
                'nickname':'2907237958',
                'content':[
                    {
                        'type':'face',
                        'data':{
                            'id':'13'
                        }
                    },
                    {
                        'type':'text',
                        'data':{
                            'text':'你说句话呀'
                        }
                    }
                ]
            }
        },
        {
            'type':'node',
            'data':{
                'user_id':'1064393873',
                'nickname':'2907237958',
                'content':[
                    {
                        'type':'text',
                        'data':{
                            'text':'你说句话呀'
                        }
                    },
                    {
                        'type':'face',
                        'data':{
                            'id':'13'
                        }
                    }
                ]
            }
        }
    ]
})

url:http://mdie.top:9700/send_private_forward_msg kw:{'json': {'user_id': '1064393873', 'messages': [{'type': 'node', 'data': {'content': [{'type': 'face', 'data': {'id': '13'}}, {'type': 'text', 'data': {'text': '你说句话呀'}}]}}, {'type': 'node', 'data': {'content': [{'type': 'text', 'data': {'text': '你说句话呀'}}, {'type': 'face', 'data': {'id': '13'}}]}}]}}
mode: post, url: send_private_forward_msg, kwargs: {'json': {'user_id': '1064393873', 'messages': [{'type': 'node', 'data': {'content': [{'type': 'face', 'data': {'id': '13'}}, {'type': 'text', 'data': {'text': '你说句话呀'}}]}}, {'type': 'node', 'data': {'content': [{'type': 'text', 'data': {'text': '你说句话呀'}}, {'type': 'face', 'data': {'id': '13'}}]}}]}}, status: failed, retcode: 200, data: None, echo: None


{'status': 'failed', 'retcode': 200, 'data': None, 'echo': None}

In [20]:
bot.SendMsg('friend',1064393873,soup.Node(soup.Image('temp/pixiv/97413473.gif')))

url:http://mdie.top:9700/send_private_forward_msg kw:{'json': {'user_id': 1064393873, 'messages': [{'type': 'node', 'data': {'user_id': '2854196310', 'nickname': 'Q群管家', 'content': [{'type': 'image', 'data': {'file': [Base64] size:177432}}]}}]}}
url:http://mdie.top:9700/get_friend_list kw:{'json': {}}
发到好友受、死[Admin(1064393873)]消息(2134816515):
[{'type': 'node', 'data': {'user_id': '2854196310', 'nickname': 'Q群管家', 'content': [{'type': 'image', 'data': {'file': [Base64] size:177432}}]}}]


{'message_id': 2134816515,
 'forward_id': 'u4lIFOgH0PQm+DF8sY82hAa3k4bF1oXbMxaggzp28ugaX++fnHQY0bYu41DDUT0p'}

In [ ]:
bot.SendMsg('friend',1064393873,
    soup.Node(
        soup.Text('标题:まとめ Pid:117224358\n作者:pottsness Uid:59336265 T\n时间:2024-03-25T00:02:25\n类型:illust 收藏比:39759/377795,10.52% 标签:\n原神:Genshin Impact\nGenshinImpact:None\n久岐忍:Kuki Shinobu\n蛍(原神):萤（原神）\nアルレッキーノ(原神):阿蕾奇诺 (原神)\nタルタリヤ:达达利亚\n放浪者(原神):Wanderer (Genshin Impact)\n原神10000users入り:原神10000收藏\n总共6张,链接时效三小时')
    ),
    soup.Node(
        soup.Image()
    ),
    soup.Node(
        soup.Image('https://i.pixiv.nl/img-original/img/2024/03/26/01/14/22/117224358_p5.jpg')
    )
)

url:http://mdie.top:9700/send_private_forward_msg kw:{'json': {'user_id': 1064393873, 'messages': [{'type': 'node', 'data': {'user_id': '2854196310', 'nickname': 'Q群管家', 'content': [{'type': 'text', 'data': {'text': '标题:まとめ Pid:117224358\n作者:pottsness Uid:59336265 T\n时间:2024-03-25T00:02:25\n类型:illust 收藏比:39759/377795,10.52% 标签:\n原神:Genshin Impact\nGenshinImpact:None\n久岐忍:Kuki Shinobu\n蛍(原神):萤（原神）\nアルレッキーノ(原神):阿蕾奇诺 (原神)\nタルタリヤ:达达利亚\n放浪者(原神):Wanderer (Genshin Impact)\n原神10000users入り:原神10000收藏\n总共6张,链接时效三小时'}}]}}, {'type': 'node', 'data': {'user_id': '2854196310', 'nickname': 'Q群管家', 'content': [{'type': 'image', 'data': {'file': [Base64] size:268551}}]}}, {'type': 'node', 'data': {'user_id': '2854196310', 'nickname': 'Q群管家', 'content': [{'type': 'image', 'data': {'file': 'https://i.pixiv.nl/img-original/img/2024/03/26/01/14/22/117224358_p5.jpg'}}]}}]}}
mode: post, url: send_private_forward_msg, kwargs: {'json': {'user_id': 1064393873, 'messages': [{'type': 'node', 'data': {'user_id': '28

{'status': 'failed', 'retcode': 200, 'data': None, 'echo': None}

In [ ]:
path = r'Z:\Home\code\qqbot\qqbot\temp\qr\202502\250225000648_qrcode.png'
bot.SendMsg('friend',1064393873,soup.Image(path))

{'type': 'image', 'file': [Base64] size:268551}
{"type": "image", "file": "base64:..."}
base64:...
url:http://mdie.top:9700/send_msg kw:{'json': {'user_id': 1064393873, 'message_type': 'private', 'message': [{'type': 'image', 'data': {'file': [Base64] size:268551}}]}}
mode: post, url: send_msg, kwargs: {'json': {'user_id': 1064393873, 'message_type': 'private', 'message': [{'type': 'image', 'data': {'file': [Base64] size:268551}}]}}, status: failed, retcode: 200, data: None, echo: None
url:http://mdie.top:9700/get_friend_list kw:{'json': {}}
发到好友受、死[Admin(1064393873)]消息(200):
[{'type': 'image', 'data': {'file': [Base64] size:268551}}]


{'status': 'failed', 'retcode': 200, 'data': None, 'echo': None}

In [29]:
path = r'Z:\Home\code\qqbot\qqbot\temp\pixiv\97413473.gif'
bot.SendMsg('friend',1064393873,soup.Image(path))

url:http://mdie.top:9700/send_msg kw:{'json': {'user_id': 1064393873, 'message_type': 'private', 'message': [{'type': 'image', 'data': {'file': [Base64] size:177432}}]}}
url:http://mdie.top:9700/get_friend_list kw:{'json': {}}
发到好友受、死[Admin(1064393873)]消息(2138842869):
[{'type': 'image', 'data': {'file': [Base64] size:177432}}]


{'message_id': 2138842869}

In [5]:
path = r"Z:\Home\code\qqbot\qqbot\temp\pixiv\63160614.gif"
bot.SendMsg('friend',1064393873,soup.Image(path))

url:http://192.168.0.2:9700/send_msg kw:{'json': {'user_id': 1064393873, 'message_type': 'private', 'message': [{'type': 'image', 'data': {'file': Base64://[size: 326364]}}]}}
url:http://192.168.0.2:9700/get_friend_list kw:{'json': {}}
发到好友受、死[Admin(1064393873)]消息(2139113546):
[{'type': 'image', 'data': {'file': Base64://[size: 326364]}}]


{'message_id': 2139113546}